In [32]:
stableStringify = (val) => {
  if (Array.isArray(val)) return `[${val.map(stableStringify).join(',')}]`;
  if (val && typeof val === 'object' && !(val instanceof Date)) {
    return `{${Object.keys(val).sort().map(key => `${JSON.stringify(key)}:${stableStringify(val[key])}`).join(',')}}`;
  }
  return JSON.stringify(val);
};
deepEqual = (a, b) => {
  if (a === b) return true;
  const aObj = a && typeof a === 'object';
  const bObj = b && typeof b === 'object';
  if (!aObj && !bObj) return Object.is(a, b);
  return stableStringify(a) === stableStringify(b);
};
formatVal = (val) => (typeof val === 'string' ? `'${val}'` : stableStringify(val));
assert = (fn, expected, label) => {
  try {
    const actual = fn();
    const pass = deepEqual(actual, expected);
    if (pass) {
      console.log('✓', label, '>>', formatVal(actual));
    } else {
      console.error('✗', label, '>> expected:', formatVal(expected), 'actual:', formatVal(actual));
    }
  } catch (err) {
    console.error('✗', label, '>> threw:', err);
  }
};

[Function: assert]

In [33]:
nameCache = new Map()
function kebabToCamel(s) {
  if (!s) return s
  let p = s.indexOf('-')
  if (p === -1) return s
  if (nameCache.has(s)) return nameCache.get(s)
  let result = s.slice(0, p)
  while (p != -1 && ++p < s.length) {
    if (s[p] == '-') continue;
    result += s[p].toUpperCase()
    if (++p < s.length)
      result += s.slice(p, (p = s.indexOf('-', p)) == -1 ? s.length : p)
  }
  nameCache.set(s, result)
  return result
}

assert(() => kebabToCamel('foo-bar'), 'fooBar', 'basic case');
assert(() => kebabToCamel('-bar'), 'Bar', 'lead single');
assert(() => kebabToCamel('bar-'), 'bar', 'trail single');
assert(() => kebabToCamel('multi-part-key'), 'multiPartKey', 'multi part');
assert(() => kebabToCamel('-'), '', 'single dash');
assert(() => kebabToCamel('--'), '', 'multi dashes only');
assert(() => kebabToCamel('--leading--dashes'), 'LeadingDashes', 'leading dashes');
assert(() => kebabToCamel('trailing--dashes-'), 'trailingDashes', 'trailing dashes');

✓ basic case >> 'fooBar'
✓ lead single >> 'Bar'
✓ trail single >> 'bar'
✓ multi part >> 'multiPartKey'
✓ single dash >> ''
✓ multi dashes only >> ''
✓ leading dashes >> 'LeadingDashes'
✓ trailing dashes >> 'trailingDashes'


In [34]:
// Returns the index of the first found char (from chars) in the s, or -1 if none found,
// then you can check s[retunedIndex] to see which char it is.
function indexFirst(s, chars, pos = 0) {
  let i, first = s.length
  for (let c of chars) if ((i = s.indexOf(c, pos)) != -1 && i < first) first = i
  return first === s.length ? -1 : first
}

assert(() => indexFirst('abcdefg', ['x', 'y', 'z']), -1, 'none found');
assert(() => indexFirst('abcdefg', ['x', 'c', 'z']), 2, 'one found');
assert(() => indexFirst('abcdefgabc', ['a', 'b', 'c'], 3), 7, 'multiple found with pos');
assert(() => indexFirst('abcdefg', ['a'], 10), -1, 'pos out of range');

✓ none found >> -1
✓ one found >> 2
✓ multiple found with pos >> 7
✓ pos out of range >> -1


In [35]:
// @WIP using +#tpl-id instead of #tpl-id to be predictable in presence of other selectors
MOD = '^', TARG = ':', TRIG = '@', STAT = '?', ADD = '+', REM = '~'
ALL = [MOD, TARG, TRIG, STAT, ADD, REM]
MODS = [MOD]

// returns { [MOD]: mods, [TARG]: [targets [it, {[MOD]: [mods...]}]...],...}
function parse(n, p='data-'.length, it = ALL) {
  let items = {}, mods = null
  while (p >= 0 && p < n.length) {
    if ((p = indexFirst(n, it, p)) == -1) { p = n.length; break }
    let t = n[p], name = ''
    if (++p < n.length) {
      let end = indexFirst(n, ALL, p)
      name = n.slice(p, p = end != -1 ? end : n.length)
    }

    let ts = items[t] ??= []
    if (t == MOD) {
      ts.push(name);
      if (p >= n.length || (it === MODS && n[p] != MOD)) return [items, p]
    } else if (p >= n.length || n[p] != MOD) {
      ts.push([name, null]);
    } else {
      [mods, p] = parse(n, p, MODS)
      ts.push([name, mods?.[MOD]])
    }
  }

  if (p < n.length) console.warn('Not everything is parsed "', n.slice(p), '" in', n)
  return [items, p]
}

assert(() => parse('data-def'), [{},8], 'empty')
assert(() => parse('data-sub:'), [{":":[["",null]]},9], 'single empty')
assert(() => parse('data-sub^mod'), [{"^":["mod"]},12], 'global mod')
assert(() => parse('data-sub^mod^'), [{"^":["mod",""]},13], '2 global mods')
assert(() => parse('data-sub^mod^@hey^foo:bar'), [{":":[["bar",null]],"@":[["hey",["foo"]]],"^":["mod",""]},25], '2 global mods and item with mod with item')
assert(() => parse('data-sub:xxx~'), [{":":[["xxx",null]],"~":[["",null]]},13], 'name and empty')
assert(() => parse('data-sub:xxx:'), [{":":[["xxx",null],["",null]]},13], 'single name')
assert(() => parse('data-sub::'), [{":":[["",null],["",null]]},10], '2 empties')
assert(() => parse('data-sub:foo^'), [{ ":": [["foo", [""]]] }, 13], 'name+empty mod')
assert(() => parse('data-sub:foo^^'), [{ ":": [["foo", ["", ""]]] }, 14], 'name+2 empty mods')
assert(() => parse('data-sub:foo-bar^bax.3'), [{ ":": [["foo-bar", ["bax.3"]]] }, 22], 'item^mod')
assert(() => parse('data-sub:foo-bar^bax.3@!something^nice'), [{ ":": [["foo-bar", ["bax.3"]]], "@": [["!something", ["nice"]]] }, 38], 'item^mod@item2^mod')

✓ empty >> [{},8]
✓ single empty >> [{":":[["",null]]},9]
✓ global mod >> [{"^":["mod"]},12]
✓ 2 global mods >> [{"^":["mod",""]},13]
✓ 2 global mods and item with mod with item >> [{":":[["bar",null]],"@":[["hey",["foo"]]],"^":["mod",""]},25]
✓ name and empty >> [{":":[["xxx",null]],"~":[["",null]]},13]
✓ single name >> [{":":[["xxx",null],["",null]]},13]
✓ 2 empties >> [{":":[["",null],["",null]]},10]
✓ name+empty mod >> [{":":[["foo",[""]]]},13]
✓ name+2 empty mods >> [{":":[["foo",["",""]]]},14]
✓ item^mod >> [{":":[["foo-bar",["bax.3"]]]},22]
✓ item^mod@item2^mod >> [{":":[["foo-bar",["bax.3"]]],"@":[["!something",["nice"]]]},38]


In [36]:

function isObjEmpty(o) {
  for (const _ in o) return false
  return true
}

WIN='_window';DOC='_document'; FRM='_form'; INTR='_interval'; TIMT='_timeout'
SPECIALS=[WIN,DOC,FRM,INTR,TIMT]
function isProp(n) {
  if (n.startsWith('#') || n.startsWith('.')) return true
  if (n.startsWith('_')) for (const s of SPECIALS) { if (n.startsWith(s, 1)) return true }
  return false
}

RETURN_THEN = [' ', '(', '{', ';', '[', '"', '\'', '\n', '\r', '\t']
SIGN_ARGS = ['dm', 'el']
TRIG_ARGS = ['dm', 'el', 'ev', 'sg', 'detail']

function compileFn(aVal, aName, args) {
  let val = '' + aVal
  var r = val.indexOf('return')
  let body = r != -1 && (r + 6 >= val.length || indexFirst(val, RETURN_THEN, r + 6) == r + 6) ? val : `return(${val})`
  body = `try{ ${body} }catch(e){ console.error('Error eval ${aName} value as function:', e.message, '>>>', ${val}); return }`
  let fn;
  try { fn = Function(...args, body) }
  catch (e) { console.error(`Error compiling ${aName} value to function:`, e.message, '>>>', val); return }
  return fn;
}

_dm = new Map()
DM = new Proxy({}, {
  get: (_, key) => _dm.get(key),
  set: (_, key, val) => { _dm.set(key, val); return true; },
  has: (_, key) => _dm.has(key),
  ownKeys: () => Array.from(_dm.keys()),
  getOwnPropertyDescriptor: (_, key) => 
    _dm.has(key) ? { value: _dm.get(key), enumerable: true, configurable: true } : undefined
});


// - data-def='{foo: {bar: "hey"}, baz: 1}' // top level fields to signals
// - data-def:foo='{bar: "hey"}' // foo signal
// - data-def:foo:baz='`js expr ${42}`' // eval expr as Function body and set to all signals
// - data-def:foo='el.Value * dm.bar' // you may use other singals and element props
function dSign(el, aName, aVal) {
  let [it, _p] = parse(aName)
  let tars = it[TARG]
  delete it[TARG]
  if (!isObjEmpty(it)) console.warn('Supports only targets but found more:', aName)
  let val = aVal ? compileFn(aVal, aName, SIGN_ARGS)(DM, el) : null
  if (tars && tars.length > 0) {
    for (const [t, mods] of tars) {
      if (mods) console.warn('Mods are not supported:', mods, 'in', aName);
      if (isProp(t)) { console.warn('Prop targets are not supported:', t, 'in', aName); continue }
      _dm.set(kebabToCamel(t), val)
    }
  } else if (!val || typeof val !== 'object') {
    console.error('Attribute', aName, 'value should contain object with signal fields, but found', aVal)
  } else {
    for (const t in val)
      _dm.set(kebabToCamel(t), val[t])
  }
}

_sign = (nam, val) => { _dm.clear(); dSign(null, nam, val); return DM }
assert(() => _sign('data-def', '{foo: {bar: "hey"}, baz: 1}'), { "baz": 1, "foo": { "bar": "hey" } }, '2 value signals')
assert(() => _sign('data-def:foo', '{bar: "hey"}'), { "foo": { "bar": "hey" } }, 'signal = value')
assert(() => _sign('data-def:foo-bar:baz'), { "baz": null, "fooBar": null }, 'signals')
assert(() => _sign('data-def:foo-bar:baz', '`Mamma Mia ${42}`'), { "baz": "Mamma Mia 42", "fooBar": "Mamma Mia 42" }, 'bonkers')

assert(() => { _dm.clear(); dSign({ "name": "John" }, 'data-def:foo', '"Hey, " + el.name'); return DM }, { "foo": "Hey, John" }, 'using el')
assert(() => { _dm.clear(); DM.name = "Noize"; dSign(null, 'data-def:greet', '"Hey, " + dm.name'); return DM }, { "name": "Noize", "greet": `Hey, Noize` }, 'using dm')

// quick check:
// fn_foo = (el) => { let n = el.name; return n }
// console.log(fn_foo({ "name": "John" }))
// fn_bar = (el) => { return (el.name) }
// console.log(fn_bar({ "name": "John" }))

✓ 2 value signals >> {"baz":1,"foo":{"bar":"hey"}}
✓ signal = value >> {"foo":{"bar":"hey"}}
✓ signals >> {"baz":null,"fooBar":null}
✓ bonkers >> {"baz":"Mamma Mia 42","fooBar":"Mamma Mia 42"}
✓ using el >> {"foo":"Hey, John"}
✓ using dm >> {"greet":"Hey, Noize","name":"Noize"}


In [ ]:
FN_CACHE = new Map();
getOrCompile = (aVal, aName, args) => {
  let fn = FN_CACHE.get(aVal)
  if (!fn) {
    fn = compileFn(aVal, aName, args)
    if (fn) FN_CACHE.set(aVal, fn)
  }
  return fn;
}

// Track all event listeners and signal handlers for cleanup
cleanupMap = new WeakMap();

function getElById(id, aName) {
  if (!id) return null
  const el = document.getElementById(id)
  if (!el) console.error(`[dmax] Error: element #${id} from ${aName} is not found`)
  return el;
};

function dSub(el, aName, aVal) {
  let [it, _p] = parse(aName)
  let targets = it[TARG]
  let triggers = it[TRIG]
  let globMods = it[MOD]
  delete it[TARG]; delete it[TRIG]; delete it[MOD];
  if (!isObjEmpty(it)) console.warn('Supports only targets, triggers, mods but found more:', aName)
  if (!aVal) { console.error('Error in ', aName, 'it requires a value but found none', aVal); return }
  let fn = getOrCompile(aVal, aName, TRIG_ARGS)
  if (!fn) return

  // Precompute appliers for targets to minimize work on each trigger
  const appliers = targets.map(tar => {
    const targetEl = tar.elemId ? getElById(tar.elemId, attr) : el;
    if (!targetEl) return null;
    if (tar.type === 'signal') {
      if (hasBracketIndex(tar.name)) {
        return res => {
          try { const resolved = resolveBracketPath(tar.name); if (resolved) set(resolved, res); } catch (e) { }
        };
      }
      return (res, isSignalInvocation) => {
        if (isSignalInvocation) set(tar.name, res); else set(tar.name, res, true);
      };
    }
    const prop = tar.propPath || null;
    return res => setProp(targetEl, prop, res);
  });



  const handler = (ev, sg, detailArg) => {
    const detailParam = ev && ev.detail && ev.detail.change ? ev.detail.change : detailArg;
    // If this invocation is signal-originated (ev is null/undefined), do not
    // pass `ev` into compiled functions — pass undefined instead; compiled
    // bodies receive the change info via the `detail` parameter.
    const isSignalInvocation = (ev == null);
    const evArg = isSignalInvocation ? undefined : ev;
    const result = fn(dmProxy, el, evArg, sg, detailParam);
    if (!appliers || appliers.length === 0) return;
    try {
      for (const ap of appliers) if (ap) ap(result, isSignalInvocation);
    } catch (e) { console.error('Applier error', e); }
  };


  // Track listeners for cleanup
  let listeners = cleanupMap.get(el);
  if (!listeners) listeners = [], cleanupMap.set(el, listeners);
  for (let t of triggers) {
    // build a decorated handler that supports modifier semantics: __prevent, __once, __debounce, __throttle, __and
    const makeDecorated = (orig, t) => {
      const { mods = {} } = t;
      const debMs = mods.debounce == null ? null : +mods.debounce;
      const thrMs = mods.throttle == null ? null : +mods.throttle;
      const once = 'once' in mods;
      const prevent = 'prevent' in mods;
      const andPath = mods.and ?? null;
      const notandPath = mods.notand ?? null;
      const guardEq = 'eq' in mods ? mods.eq : undefined;
      const guardNe = 'ne' in mods ? mods.ne : undefined;
      const guardGt = 'gt' in mods ? +mods.gt : undefined;
      const guardGe = 'ge' in mods ? +mods.ge : undefined;
      const guardLt = 'lt' in mods ? +mods.lt : undefined;
      const guardLe = 'le' in mods ? +mods.le : undefined;
      let timer = null;
      let last = 0;
      const decorated = function (ev, sg, detailArg) {
        if (prevent) ev?.preventDefault?.();
        const run = () => {
          if (andPath && !get(String(andPath))) return;
          if (notandPath && get(String(notandPath))) return;
          // Get subject value, apply negation if trigger is negated
          let subjectVal = t.type === 'signal' ? get(String(t.name)) : ev?.detail?.ms;
          if (t.negated && t.type === 'signal') subjectVal = !subjectVal;
          if (guardEq !== undefined && String(subjectVal) !== String(guardEq)) return;
          if (guardNe !== undefined && String(subjectVal) === String(guardNe)) return;
          if (guardGt !== undefined && !(+subjectVal > guardGt)) return;
          if (guardGe !== undefined && !(+subjectVal >= guardGe)) return;
          if (guardLt !== undefined && !(+subjectVal < guardLt)) return;
          if (guardLe !== undefined && !(+subjectVal <= guardLe)) return;
          try { orig(ev, sg, detailArg); } catch (e) { console.error('Handler error', e); }
          if (once && decorated && decorated.remove) decorated.remove();
        };
        if (debMs != null) { clearTimeout(timer); timer = setTimeout(run, debMs); return; }
        if (thrMs != null) { const now = Date.now(); if (now - last >= thrMs) { last = now; run(); } return; }
        run();
      };
      return decorated;
    };

    if (t.type === 'signal') {
      const rootRaw = String(t.name).split('.')[0];
      const rootBase = (rootRaw.indexOf('[') !== -1) ? rootRaw.split('[')[0] : rootRaw;
      const root = toCamel(rootBase);
      const rootAlias = sigKey(rootBase);
      if (!subs.has(root)) subs.set(root, []);
      if (rootAlias !== root && !subs.has(rootAlias)) subs.set(rootAlias, []);
      const dec = makeDecorated(handler, t);
      // allow removal for __once and other removals; handle both legacy function entries and object entries
      dec.remove = () => {
        const arr = subs.get(root) || [];
        subs.set(root, arr.filter(it => (typeof it === 'function' ? it !== dec : it.fn !== dec)));
        if (rootAlias !== root) { const arr2 = subs.get(rootAlias) || []; subs.set(rootAlias, arr2.filter(it => (typeof it === 'function' ? it !== dec : it.fn !== dec))); }
      };
      // support __shape modifier: register subscription object with optional childPath
      let entry = dec;
      if (t.mods && ('shape' in t.mods || 'content' in t.mods)) {
        const sval = (t.mods && 'shape' in t.mods) ? ((t.mods.shape === true || t.mods.shape === 1) ? null : (t.mods.shape != null ? String(t.mods.shape) : null)) : null;
        entry = { fn: dec, mode: ('shape' in t.mods ? 'shape' : 'content'), childPath: sval };
      }
      // If trigger uses bracket-index indirection, wrap the handler so it only fires when the resolved path matches
      if (hasBracketIndex(t.name)) {
        const wrapper = function (ev, p, detailArg) {
          try {
            const resolved = resolveBracketPath(t.name);
            // debug trace
            console.log('WRAPPER', t.name, 'resolved->', resolved, 'p->', p);
            // if invoked because of index/root change (p is one of the extra roots) we should call handler
            const extraRoots = Array.from(collectBracketRoots(t.name));
            if (p && extraRoots.includes(p)) {
              return dec(ev, p, detailArg);
            }
            if (!p) return;
            // if mutation path equals or is under resolved path, invoke
            if (resolved && (p === resolved || p.indexOf(resolved + '.') === 0)) {
              return dec(ev, p, detailArg);
            }
          } catch (e) { }
        };
        const wrappedEntry = { fn: wrapper, mode: (t.mods && ('shape' in t.mods) ? 'shape' : 'content'), childPath: null };
        subs.get(root).push(wrappedEntry);
        if (rootAlias !== root) subs.get(rootAlias).push(wrappedEntry);
        // also subscribe wrapper to bracket roots (idx signals)
        const extraRoots = collectBracketRoots(t.name);
        for (const er of extraRoots) { if (er !== root) { if (!subs.has(er)) subs.set(er, []); subs.get(er).push(wrappedEntry); } }
      } else {
        subs.get(root).push(entry);
        if (rootAlias !== root) subs.get(rootAlias).push(entry);
      }
      listeners.push({ type: 'signal', root, handler: dec });
      if (rootAlias !== root) listeners.push({ type: 'signal', root: rootAlias, handler: dec });
      if (!t.mods?.notimmediate) dec(undefined, t.name);
      continue;
    }

    // Event triggers
    // Resolve target element: explicit id (`#id`) or current element
    let targetEl = null;
    let eventName = t.eventName || null;

    // Case: explicit element id — no legacy mappings, require real element id
    if (t.elemId) {
      targetEl = getElById(t.elemId, attr);
      if (!targetEl) { console.error('Element not found for id:', t.elemId); }
    }

    // Case: new syntax uses `.window` or `.interval` as eventName while isCurr is true
    // Handle special triggers produced by the `_` token: @_window, @_document, @_form, @_interval, @_delay
    if (t.type === 'special') {
      const name = t.name;
      if (name === 'window') {
        const ev = (t.arg && String(t.arg)) || 'resize';
        window.addEventListener(ev, handler);
        listeners.push({ type: 'event', targetEl: window, eventName: ev, handler });
        if (t.mods?.immediate) handler();
        continue;
      }
      if (name === 'document') {
        const ev = (t.arg && String(t.arg)) || 'visibilitychange';
        document.addEventListener(ev, handler);
        listeners.push({ type: 'event', targetEl: document, eventName: ev, handler });
        if (t.mods?.immediate) handler();
        continue;
      }
      if (name === 'interval') {
        const ms = parseInt(t.arg) || 1000;
        const tick = () => {
          try { handler(new CustomEvent('interval', { detail: { ms } })); } catch (e) { }
        };
        const id = setInterval(tick, ms);
        listeners.push({ type: 'interval', id });
        if (t.mods?.immediate) tick();
        continue;
      }
      if (name === 'delay') {
        const ms = parseInt(t.arg) || 0;
        const id = setTimeout(() => {
          try { handler(new CustomEvent('delay', { detail: { ms } })); } catch (e) { }
        }, ms);
        listeners.push({ type: 'timeout', id });
        continue;
      }
      if (name === 'form') {
        // bind to closest form ancestor
        const formEl = el && el.closest ? el.closest('form') : null;
        if (formEl) {
          const ev = (t.arg && String(t.arg)) || 'submit';
          formEl.addEventListener(ev, handler);
          listeners.push({ type: 'event', targetEl: formEl, eventName: ev, handler });
          if (t.mods?.immediate) handler();
        }
        continue;
      }
    }

    // If still no targetEl, fall back to current element
    if (!targetEl) targetEl = el;
    // Determine final event name
    const finalEvent = eventName || getAutoEvent(targetEl);
    if (targetEl === null) { console.error('Element not found:', t.elemId); continue; }
    // decorate handler with modifiers
    const dec = (function (orig, tt) {
      const mods = tt.mods || {};
      const decorated = makeDecorated(orig, tt);
      decorated.remove = () => { try { targetEl.removeEventListener(finalEvent, decorated); } catch (e) { } };
      return decorated;
    })(handler, t);
    targetEl.addEventListener(finalEvent, dec);
    listeners.push({ type: 'event', targetEl, eventName: finalEvent, handler: dec });
    if (t.mods?.immediate) dec();
  }
  if (triggers.length === 0) handler();
}


[Function: getElById]